In [ ]:
import time
import numpy as np
import pandas as pd
import torch

import torch.optim as optim

In [ ]:
from src.utils_data import M4TS
from src.utils_data import get_m4_all_series

from src.utils_config import ModelConfig
from src.utils_visualization import plot_prediction

from src.utils_train import train

In [ ]:
mc = ModelConfig(config_file='configs/config_m4quarterly.yaml',
                 root_dir='./results/')

all_series = get_m4_all_series(mc=mc, data='train')
train(mc, all_series)

# model = ESRNN(mc)
# print("\n Model parameters: \n", model.rnn.parameters, "\n")